In [12]:
bam_paths = Sys.glob("/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/merged_by_celltype/*.bam")
bam_ontologies = lapply(lapply(bam_paths,basename),tools::file_path_sans_ext)


[[1]]
[1] "Astro"

[[2]]
[1] "doublet"

[[3]]
[1] "Endo"

[[4]]
[1] "L23_IT"

[[5]]
[1] "L56_NP"

[[6]]
[1] "L5_ET"

[[7]]
[1] "L5_IT"

[[8]]
[1] "L6b"

[[9]]
[1] "L6_CT"

[[10]]
[1] "L6_IT"

[[11]]
[1] "L6_IT_Car3"

[[12]]
[1] "Lamp5"

[[13]]
[1] "Low_Quality"

[[14]]
[1] "Macrophage"

[[15]]
[1] "Oligo"

[[16]]
[1] "OPC"

[[17]]
[1] "SMC"

[[18]]
[1] "Sncg"

[[19]]
[1] "Sst"

[[20]]
[1] "Vip"

[[21]]
[1] "VLMC"

In [19]:
library(Gviz)
library(GenomicFeatures)
library(data.table)

#support for plus/minus strand bam plotting from:
#https://support.bioconductor.org/p/56047/
library(Rsamtools)
strandedBamImport <- function (file, selection) {
    if (!file.exists(paste(file, "bai", sep = ".")))
        stop("Unable to find index for BAM file '", file, "'. You can
build an index using the following command:\n\t",
             "library(Rsamtools)\n\tindexBam(\"", file, "\")")
    sinfo <- scanBamHeader(file)[[1]]
    res <- if (!as.character(seqnames(selection)[1]) %in%
               names(sinfo$targets)) {
        mcols(selection) <- DataFrame(score = 0)
        selection
    }else {
        param <- ScanBamParam(what = c("pos", "qwidth", "strand"),
                              which = selection, flag =
scanBamFlag(isUnmappedQuery = FALSE))
        x <- scanBam(file, param = param)[[1]]
        gr <- GRanges(strand=x[["strand"]], ranges=IRanges(x[["pos"]],
width = x[["qwidth"]]), seqnames=seqnames(selection)[1])
        grs <- split(gr, strand(gr))
        cov <- lapply(grs[c("+", "-")], function(y)
coverage(ranges(y),
width=end(selection)))
        pos <- sort(unique(unlist(lapply(cov, function(y) c(start(y),
end(y))))))
        if(length(pos)==0){
            mcols(selection) <- DataFrame(plus=0, minus=0)
            selection
        }else{
            GRanges(seqnames = seqnames(selection)[1],
ranges=IRanges(start=head(pos, -1), end=tail(pos, -1)),
                    plus=as.numeric(cov[["+"]][head(pos, -1)]),
                    minus=-as.numeric(cov[["-"]][head(pos, -1)]))
        }
    }
    return(res)
}
                                         

#args = commandArgs(trailingOnly=TRUE)

#print(args)

gene_name = "Timp3"

txdb_path = "txdb.mm10.sqlite"
bed_path = "MERFISH_genes.bed"
srrs_path = "/oak/stanford/groups/horence/rob/isoform_localizations/SRRS/final_outputs/20220103_merfish.csv"
bam_stem = "/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/merged_by_celltype/"
                                         
bam_paths = Sys.glob(paste(bam_stem,"*.bam",sep=""))
bam_ontologies = lapply(lapply(bam_paths,basename),tools::file_path_sans_ext)
                                         
#find which cell-types to plot from SRRS
srrs_df = fread(srrs_path)
srrs_df = srrs_df[gene == gene_name] #filter to results from the gene of interest
srrs_df = srrs_df[bh_p < 0.05] #filter by bh_p
srrs_df = srrs_df[order(med_score)] #order by effect size
srrs_df = srrs_df[ontology %in% bam_ontologies] #keep only ontologies which have corresponding bams

n_onts_to_plot = 40
                                         
if(nrow(srrs_df) == 0){
    print("No matching cell-types")
    stop()
} else if (nrow(srrs_df) <= n_onts_to_plot){
    #plot all cell types
    num_ontologies = nrow(srrs_df)
    cell_types = srrs_df$ontology
    effect_sizes = srrs_df$med_score
} else {
    #plot top n/2 and bottom n/2 cell-types by med score (effect size)
    n = n_onts_to_plot/2
    cell_types = c(head(srrs_df,n)$ontology, tail(srrs_df,n)$ontology)
    effect_sizes = c(head(srrs_df,n)$med_score, tail(srrs_df,n)$med_score)
}

                                         
gene_locs = fread(bed_path)
gene_row = gene_locs[gene == gene_name]

chr = gene_row[, get("#chr")]
window_start = gene_row$start
window_end   = gene_row$end
outfile = paste(gene_name,".png",sep="")
tracks = list()
                                         
####################################
#                                  #
#                                  #
#            Gene Track            #
#                                  #
#                                  #
####################################
txdb = loadDb(txdb_path)

geneTrack = GeneRegionTrack(
    range = txdb,
    chromosome = chr,
    start = window_start,
    end = window_end,
    name = gene_name,
)

geneTrack <- geneTrack[
    feature(geneTrack) %in% c("CDS", "UTR")
]
tracks = append(tracks,geneTrack)

gTrack <- GenomeAxisTrack()                                         
tracks = append(tracks,gTrack)

####################################
#                                  #
#                                  #
#            Data Track            #
#                                  #
#                                  #
####################################
for(i in 1:num_ontologies){
    ct = cell_types[[i]]
    ct = sub(" ","_",ct)
    
    effect_size = effect_sizes[[i]]
    effect_size = format(round(effect_size, 2), nsmall = 2)
    
    cov_track = DataTrack(
        range = paste(bam_stem,ct,".bam",sep=""),
        name = paste(ct,effect_size,sep="\n"),
        chromosome = chr,
        importFunction=strandedBamImport,
        stream=TRUE,
    )
    tracks = append(tracks,cov_track)
}

####################################
#                                  #
#                                  #
#            Plotting              #
#                                  #
#                                  #
####################################
png(outfile, width = 800, height = 800)

plotTracks(
    tracks,
    type = "h",
    from = window_start, 
    to = window_end,
    thinBoxFeature = "UTR",
    collapse = F,
    transcriptAnnotation = "gene",
    col=c("red","blue"),
    groups=c("+", "-"),
    fontcolor.exon = "black",
    legend = F,
    cex = 0.6
)

dev.off()

png 
  2

In [37]:
bam_stem = "/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/merged_by_celltype/"
paste(bam_stem,ct,".bam",sep="")

[1] "/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/merged_by_celltype/L6_CT.bam"

In [50]:
p.adjust

function (p, method = p.adjust.methods, n = length(p)) 
{
    method <- match.arg(method)
    if (method == "fdr") 
        method <- "BH"
    nm <- names(p)
    p <- as.numeric(p)
    p0 <- setNames(p, nm)
    if (all(nna <- !is.na(p))) 
        nna <- TRUE
    p <- p[nna]
    lp <- length(p)
    stopifnot(n >= lp)
    if (n <= 1) 
        return(p0)
    if (n == 2 && method == "hommel") 
        method <- "hochberg"
    p0[nna] <- switch(method, bonferroni = pmin(1, n * p), holm = {
        i <- seq_len(lp)
        o <- order(p)
        ro <- order(o)
        pmin(1, cummax((n + 1L - i) * p[o]))[ro]
    }, hommel = {
        if (n > lp) p <- c(p, rep.int(1, n - lp))
        i <- seq_len(n)
        o <- order(p)
        p <- p[o]
        ro <- order(o)
        q <- pa <- rep.int(min(n * p/i), n)
        for (j in (n - 1L):2L) {
            ij <- seq_len(n - j + 1L)
            i2 <- (n - j + 2L):n
            q1 <- min(j * p[i2]/(2L:j))
            q[ij] <- pmin(j * p[ij], q1)
            q[i2] <- q[n - j + 1L]
            pa <- pmax(pa, q)
        }
        pmax(pa, p)[if (lp < n) ro[1L:lp] else ro]
    }, hochberg = {
        i <- lp:1L
        o <- order(p, decreasing = TRUE)
        ro <- order(o)
        pmin(1, cummin((n + 1L - i) * p[o]))[ro]
    }, BH = {
        i <- lp:1L
        o <- order(p, decreasing = TRUE)
        ro <- order(o)
        pmin(1, cummin(n/i * p[o]))[ro]
    }, BY = {
        i <- lp:1L
        o <- order(p, decreasing = TRUE)
        ro <- order(o)
        q <- sum(1/(1L:n))
        pmin(1, cummin(q * n/i * p[o]))[ro]
    }, none = p)
    p0
}
<bytecode: 0x36f6f910>
<environment: namespace:stats>